In [1]:
%load_ext autoreload
%autoreload 2

import jax.numpy as jnp
import numpy as np
import random
import mujoco_py
import gym
import jax.random as jrandom
from sklearn import metrics
import matplotlib.pyplot as plt

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

from replay_buffer import ReplayBuffer
from constants import SEED
from policy import evaluate_policy
from pilco_utils import train_transition_models
from rff import phi_X
from blr import BLR

Import error. Trying to rebuild mujoco_py.
Compiling /Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/mujoco_py/cymj.pyx
running build_ext
/usr/local/opt/llvm/bin/clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -iwithsysroot/System/Library/Frameworks/System.framework/PrivateHeaders -iwithsysroot/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.7/Headers -I/usr/local/opt/llvm/include -DONMAC -I/Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/mujoco_py -I/Users/damso/.mujoco/mujoco210/include -I/Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/numpy/core/include -I/Users/damso/Year4/FYP/pipps/venv/include -I/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.7/include/python3.7m -c /Users/damso

In file included from /Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/mujoco_py/cymj.c:62:
/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.7/include/python3.7m/Python.h:25:10: fatal error: 'stdio.h' file not found
#include <stdio.h>
         ^~~~~~~~~
1 error generated.


CompileError: command '/usr/local/opt/llvm/bin/clang' failed with exit code 1

In [107]:
env = gym.make('InvertedPendulum-v2')
state_dim = env.observation_space.shape[0]
num_actions = env.action_space.n

ACTIONS = np.array([0, 1])

Import error. Trying to rebuild mujoco_py.
running build_ext
/usr/local/bin/gcc-9 -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -iwithsysroot/System/Library/Frameworks/System.framework/PrivateHeaders -iwithsysroot/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.7/Headers -DONMAC -I/Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/mujoco_py -I/Users/damso/.mujoco/mujoco210/include -I/Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/numpy/core/include -I/Users/damso/Year4/FYP/pipps/venv/include -I/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.7/include/python3.7m -c /Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/mujoco_py/cymj.c -o /Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_37_macextensionbuilder/temp.macosx-10.14-x86_64-3.7/Users/damso/Year4/FYP/pipps/

CompileError: command '/usr/local/bin/gcc-9' failed: No such file or directory: '/usr/local/bin/gcc-9'

### Run random policy to collect training data

In [ ]:
INIT_EPISODES = 50
BUFFER_CAPACITY = 5000
HORIZON = 200

replay_buffer = ReplayBuffer(capacity=BUFFER_CAPACITY)

# Collect transitions to train transition model
for ep in range(INIT_EPISODES):
    cur_state = env.reset()
    done = False
    
    transitions = []
    for t in range(HORIZON):
        # action = np.array([int(random.uniform(-3., 3.))])
        action = np.array([0.32], dtype=np.float32)
        prev_state = cur_state

        cur_state, _, done, _ = env.step(action)
        replay_buffer.push(prev_state, action, cur_state, t + 1)

        if done:
            break

AssertionError: array([0.32], dtype=float32) (<class 'numpy.ndarray'>) invalid

### Evaluate policy and compute gradients

In [ ]:
theta = jnp.ones((state_dim + 1,))
N = 100
HORIZON = 100
NUM_FEATURES = 2000

model_d1 = BLR(NUM_FEATURES, alpha=0.3, beta=1.)
model_d2 = BLR(NUM_FEATURES, alpha=0.3, beta=1.)
model_d3 = BLR(NUM_FEATURES, alpha=0.3, beta=1.)
model_d4 = BLR(NUM_FEATURES, alpha=0.3, beta=1.)

train_transition_models(SEED, replay_buffer, model_d1, model_d2, model_d3, model_d4)

evaluate_policy(theta, [model_d1, model_d2, model_d3, model_d4], env, N, HORIZON)

ValueError: vmap in_axes specification must be a tree prefix of the corresponding value, got specification (None, None, None, None, None) for value tree PyTreeDef((*, [*, *, *, *], *, *)).

### Collect and organise training and testing data

In [ ]:
train_X, train_y, test_X, test_y = replay_buffer.get_train_test_arrays()

train_X_d1 = train_X[:, :2]
train_X_d2 = train_X[:, 2:4]
train_X_d3 = train_X[:, 4:6]
train_X_d4 = train_X[:, 6:8]
test_X_d1 = test_X[:, :2]
test_X_d2 = test_X[:, 2:4]
test_X_d3 = test_X[:, 4:6]
test_X_d4 = test_X[:, 6:8]

train_y_d1 = train_y[:, 0]
train_y_d2 = train_y[:, 1]
train_y_d3 = train_y[:, 2]
train_y_d4 = train_y[:, 3]
test_y_d1 = test_y[:, 0]
test_y_d2 = test_y[:, 1]
test_y_d3 = test_y[:, 2]
test_y_d4 = test_y[:, 3]

### Apply Random Fourier Features

In [ ]:
NUM_FEATURES = 2000

omega = jrandom.normal(key=SEED, shape=(NUM_FEATURES, 2))
phi = jrandom.uniform(key=SEED, minval=0, maxval=2 * jnp.pi, shape=(NUM_FEATURES, 1))
lengthscales = jnp.full((NUM_FEATURES, 1), 1.)
coefs = jnp.full_like(lengthscales, 1.)

phi_X_train_d1 = phi_X(train_X_d1, NUM_FEATURES, lengthscales, coefs, omega, phi)
phi_X_test_d1 = phi_X(test_X_d1, NUM_FEATURES, lengthscales, coefs, omega, phi)
phi_X_train_d2 = phi_X(train_X_d2, NUM_FEATURES, lengthscales, coefs, omega, phi)
phi_X_test_d2 = phi_X(test_X_d2, NUM_FEATURES, lengthscales, coefs, omega, phi)
phi_X_train_d3 = phi_X(train_X_d3, NUM_FEATURES, lengthscales, coefs, omega, phi)
phi_X_test_d3 = phi_X(test_X_d3, NUM_FEATURES, lengthscales, coefs, omega, phi)
phi_X_train_d4 = phi_X(train_X_d4, NUM_FEATURES, lengthscales, coefs, omega, phi)
phi_X_test_d4 = phi_X(test_X_d4, NUM_FEATURES, lengthscales, coefs, omega, phi)

### Train RFF transition model

In [ ]:

model_d1 = BLR(NUM_FEATURES, alpha=0.3, beta=1.)
model_d2 = BLR(NUM_FEATURES, alpha=0.3, beta=1.)
model_d3 = BLR(NUM_FEATURES, alpha=0.3, beta=1.)
model_d4 = BLR(NUM_FEATURES, alpha=0.3, beta=1.)

model_d1.posterior(phi_X_train_d1, train_y_d1)
model_d2.posterior(phi_X_train_d2, train_y_d2)
model_d3.posterior(phi_X_train_d3, train_y_d3)
model_d4.posterior(phi_X_train_d4, train_y_d4)

d1_preds = np.zeros_like(test_y_d1)
d2_preds = np.zeros_like(test_y_d2)
d3_preds = np.zeros_like(test_y_d3)
d4_preds = np.zeros_like(test_y_d4)

for i in range(test_y_d1.shape[0]):
    d1_preds = model_d1.predict(phi_X_test_d1).mean()
    d2_preds = model_d2.predict(phi_X_test_d2).mean()
    d3_preds = model_d3.predict(phi_X_test_d3).mean()
    d4_preds = model_d4.predict(phi_X_test_d4).mean()



print(f"MSE d1: {metrics.mean_absolute_error(test_y_d1, d1_preds)}")
print(f"MSE d2: {metrics.mean_absolute_error(test_y_d2, d2_preds)}")
print(f"MSE d3: {metrics.mean_absolute_error(test_y_d3, d3_preds)}")
print(f"MSE d4: {metrics.mean_absolute_error(test_y_d4, d4_preds)}")

MSE d1: 0.0005051717684330179
MSE d2: 0.1945817729806484
MSE d3: 0.010503681856162989
MSE d4: 0.29280206051311997


### Train Gaussian process transition model

In [ ]:

gp_d1 = GaussianProcessRegressor(kernel=RBF(length_scale_bounds=(1e-4, 1e3)), n_restarts_optimizer=9)
gp_d2 = GaussianProcessRegressor(kernel=RBF(length_scale_bounds=(1e-4, 1e3)), n_restarts_optimizer=9)
gp_d3 = GaussianProcessRegressor(kernel=RBF(length_scale_bounds=(1e-4, 1e3)), n_restarts_optimizer=9)
gp_d4 = GaussianProcessRegressor(kernel=RBF(length_scale_bounds=(1e-4, 1e3)), n_restarts_optimizer=9)

gp_d1.fit(train_X_d1, train_y_d1)
gp_d2.fit(train_X_d2, train_y_d2)
gp_d3.fit(train_X_d3, train_y_d3)
gp_d4.fit(train_X_d4, train_y_d4)

d1_preds_gp = np.zeros_like(test_y_d1)
d2_preds_gp = np.zeros_like(test_y_d2)
d3_preds_gp = np.zeros_like(test_y_d3)
d4_preds_gp = np.zeros_like(test_y_d4)

d1_std_gp = np.zeros_like(test_y_d1)
d2_std_gp = np.zeros_like(test_y_d2)
d3_std_gp = np.zeros_like(test_y_d3)
d4_std_gp = np.zeros_like(test_y_d4)

for i in range(test_y_d1.shape[0]):
    d1_preds_gp, d1_std_gp = gp_d1.predict(test_X_d1, return_std=True)
    d2_preds_gp, d2_std_gp = gp_d2.predict(test_X_d2, return_std=True)
    d3_preds_gp, d3_std_gp = gp_d3.predict(test_X_d3, return_std=True)
    d4_preds_gp, d4_std_gp = gp_d4.predict(test_X_d4, return_std=True)

print(f"MSE d1: {metrics.mean_absolute_error(test_y_d1, d1_preds_gp)}, kernel: {gp_d1.kernel_}")
print(f"MSE d2: {metrics.mean_absolute_error(test_y_d2, d2_preds_gp)}, kernel: {gp_d2.kernel_}")
print(f"MSE d3: {metrics.mean_absolute_error(test_y_d3, d3_preds_gp)}, kernel: {gp_d3.kernel_}")
print(f"MSE d4: {metrics.mean_absolute_error(test_y_d4, d4_preds_gp)}, kernel: {gp_d4.kernel_}")

/Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/sklearn/gaussian_process/_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/damso/Year4/FYP/pipps/venv/lib/python3.7/site-packages/sklearn/gaussian_process/_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


MSE d1: 1.27629495939378e-07, kernel: RBF(length_scale=135)
MSE d2: 0.20035772283992484, kernel: RBF(length_scale=0.00346)
MSE d3: 0.010510766074957068, kernel: RBF(length_scale=551)
MSE d4: 0.29280441122477135, kernel: RBF(length_scale=1)


### Particle-based PILCO

In [ ]:
MAX_EPISODES = 500
key = SEED
for ep in range(MAX_EPISODES):
    key, subkey = jrandom.split(key)
    # Train model with all available data
    train_transition_models(
        replay_buffer, model_d1, model_d2, model_d3, model_d4
    )

    # Evaluate the current policy using learned model
    cur_obj = get_policy_evaluation(

    )

    policy_grad = get_policy_grad(

    )

    # Roll-out policy online for 1 episode and add to replay buffer
    

